In [1]:
# NeuroTouch+ Full Multimodal ML Pipeline (EEG + HRV + Finger + Emotion + Keystroke + Self-report)

# STEP 1: Install Requirements
!pip install mne pandas numpy matplotlib seaborn deepface transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=b480b8b9395e1c22f39d01f8250bfb0c75978a357829f89841378ef1d49882f9
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [15]:
import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from glob import glob
from deepface import DeepFace
from transformers import pipeline

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# STEP 1: Load HRV entries first
hrv_files = glob("/content/drive/MyDrive/apple_health_HRV/*.csv")
print("✅ HRV summary files found:", hrv_files)

records = []
for f in hrv_files:
    try:
        df = pd.read_csv(f, header=None)
        recorded_row = df[df[0].str.contains("Recorded Date", na=False)]
        timestamp = pd.to_datetime(recorded_row.iloc[0, 1], errors="coerce") if not recorded_row.empty else None
        class_row = df[df[0].str.contains("Classification", na=False)]
        classification = class_row.iloc[0, 1] if not class_row.empty else None

        if timestamp is not None:
            records.append({
                "timestamp": timestamp,
                "classification": classification,
                "source_file": f.split("/")[-1]
            })

    except Exception as e:
        print(f"⚠️ Error reading {f}: {e}")

df_hrv = pd.DataFrame(records)
df_hrv = df_hrv.sort_values("timestamp").reset_index(drop=True)
print("✅ Parsed HRV entries:", df_hrv.shape)

# STEP 2: Use first HRV timestamp as EEG base time
if df_hrv.empty:
    raise ValueError("❌ No valid HRV entries found.")

eeg_base_time = df_hrv["timestamp"].min().replace(tzinfo=None)
print("🧠 EEG synthetic start time set to:", eeg_base_time)

✅ HRV summary files found: ['/content/drive/MyDrive/apple_health_HRV/ecg_2024-05-01.csv', '/content/drive/MyDrive/apple_health_HRV/ecg_2024-10-23.csv', '/content/drive/MyDrive/apple_health_HRV/ecg_2024-05-02.csv', '/content/drive/MyDrive/apple_health_HRV/ecg_2024-08-05.csv', '/content/drive/MyDrive/apple_health_HRV/ecg_2024-05-29.csv']
✅ Parsed HRV entries: (5, 3)
🧠 EEG synthetic start time set to: 2024-05-01 11:28:37


In [18]:
# STEP 3: Load EEG and simulate timestamps
eeg_dir = "/content/drive/MyDrive/sub-NDARAB055BPR-NDARAB348EWR"
eeg_files = [f for f in os.listdir(eeg_dir) if f.endswith(".set")]
print("✅ Found EEG files:", eeg_files)

from tqdm import tqdm
all_eeg_data = []
current_time = eeg_base_time

for filename in tqdm(eeg_files):
    eeg_path = os.path.join(eeg_dir, filename)
    raw = mne.io.read_raw_eeglab(eeg_path, preload=True)
    raw.filter(1., 40.)

    psds, freqs = raw.compute_psd(fmin=1, fmax=40).get_data(return_freqs=True)
    alpha = psds[:, (freqs >= 8) & (freqs <= 13)].mean(axis=1)
    beta  = psds[:, (freqs >= 13) & (freqs <= 30)].mean(axis=1)
    theta = psds[:, (freqs >= 4) & (freqs <= 7)].mean(axis=1)

    n_samples = len(alpha)
    timestamps = pd.date_range(start=current_time, periods=n_samples, freq="1S")
    current_time = timestamps[-1] + timedelta(minutes=1)  # avoid overlap between files

    df_eeg = pd.DataFrame({
        "timestamp": timestamps,
        "alpha_power": alpha,
        "beta_power": beta,
        "theta_power": theta,
        "source_file": filename
    })
    all_eeg_data.append(df_eeg)

df_all_eeg = pd.concat(all_eeg_data).reset_index(drop=True)
print("✅ EEG final shape:", df_all_eeg.shape)

✅ Found EEG files: ['sub-NDARAB055BPR_task-DespicableMe_eeg.set', 'sub-NDARAB055BPR_task-DiaryOfAWimpyKid_eeg.set', 'sub-NDARAB055BPR_task-FunwithFractals_eeg.set', 'sub-NDARAB055BPR_task-RestingState_eeg.set', 'sub-NDARAB055BPR_task-ThePresent_eeg.set', 'sub-NDARAB055BPR_task-contrastChangeDetection_run-1_eeg.set', 'sub-NDARAB055BPR_task-contrastChangeDetection_run-2_eeg.set', 'sub-NDARAB055BPR_task-contrastChangeDetection_run-3_eeg.set', 'sub-NDARAB055BPR_task-seqLearning8target_eeg.set', 'sub-NDARAB055BPR_task-surroundSupp_run-1_eeg.set', 'sub-NDARAB055BPR_task-surroundSupp_run-2_eeg.set', 'sub-NDARAB348EWR_task-contrastChangeDetection_run-1_eeg.set', 'sub-NDARAB348EWR_task-contrastChangeDetection_run-2_eeg.set', 'sub-NDARAB348EWR_task-contrastChangeDetection_run-3_eeg.set', 'sub-NDARAB348EWR_task-DespicableMe_eeg.set', 'sub-NDARAB348EWR_task-FunwithFractals_eeg.set', 'sub-NDARAB348EWR_task-DiaryOfAWimpyKid_eeg.set', 'sub-NDARAB348EWR_task-RestingState_eeg.set', 'sub-NDARAB348EWR_ta

  0%|          | 0/22 [00:00<?, ?it/s]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.5s


Effective window size : 4.096 (s)


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.5s


Effective window size : 4.096 (s)


100%|██████████| 22/22 [00:56<00:00,  2.57s/it]

✅ EEG final shape: (2838, 5)


In [19]:
# STEP 4: Merge EEG + HRV (nearest timestamp)
df_all_eeg["timestamp_rounded"] = df_all_eeg["timestamp"].dt.round("min")
df_hrv["timestamp_rounded"] = df_hrv["timestamp"].dt.tz_localize(None).dt.round("min")

df_merged = pd.merge_asof(
    df_all_eeg.sort_values("timestamp_rounded"),
    df_hrv.sort_values("timestamp_rounded"),
    on="timestamp_rounded",
    direction="nearest",
    tolerance=pd.Timedelta("5min")
)

print("✅ Merged EEG + HRV shape:", df_merged.shape)
df_merged.head()

✅ Merged EEG + HRV shape: (2838, 9)


,timestamp_x,alpha_power,beta_power,theta_power,source_file_x,timestamp_rounded,timestamp_y,classification,source_file_y
0,2024-05-01 11:28:37,6.289140e-12,2.070559e-12,1.882640e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv
1,2024-05-01 11:29:06,6.296306e-12,2.071560e-12,1.884595e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv
2,2024-05-01 11:29:07,6.294744e-12,2.074916e-12,1.885256e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv
3,2024-05-01 11:29:08,6.301253e-12,2.071550e-12,1.884954e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv
4,2024-05-01 11:29:09,6.041612e-12,1.998642e-12,1.852315e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv


In [20]:
np.random.seed(42)  # For reproducibility

n = len(df_merged)

# Simulated finger usage
df_merged["tap_speed"] = np.random.normal(0.5, 0.1, n).clip(0.3, 0.8)
df_merged["swipe_velocity"] = np.random.normal(1.0, 0.3, n).clip(0.5, 2.0)
df_merged["tap_pressure"] = np.random.normal(0.8, 0.2, n).clip(0.5, 1.0)
df_merged["screen_time_min"] = np.random.randint(10, 300, n)
df_merged["unlock_freq"] = np.random.randint(5, 60, n)

# Simulated keystroke features
df_merged["avg_key_hold_time"] = np.random.normal(0.2, 0.05, n)
df_merged["avg_key_interval"] = np.random.normal(0.25, 0.08, n)
df_merged["typo_rate"] = np.random.normal(0.05, 0.03, n).clip(0, 0.2)

# Simulated valence and emotion
valence = np.random.normal(0, 1, n)
df_merged["valence_score"] = valence
df_merged["emotion_label"] = np.where(valence > 0.5, "happy",
                               np.where(valence < -0.5, "sad", "neutral"))

# Simulated self-reports
df_merged["mood_score"] = np.random.randint(1, 11, n)
df_merged["fatigue_level"] = np.random.randint(1, 4, n)
df_merged["attention_level"] = np.random.randint(1, 11, n)
df_merged["productivity"] = np.random.randint(1, 11, n)

print("✅ Simulated features added:", df_merged.shape)
df_merged.head()


✅ Simulated features added: (2838, 23)


,timestamp_x,alpha_power,beta_power,theta_power,source_file_x,timestamp_rounded,timestamp_y,classification,source_file_y,tap_speed,...,unlock_freq,avg_key_hold_time,avg_key_interval,typo_rate,valence_score,emotion_label,mood_score,fatigue_level,attention_level,productivity
0,2024-05-01 11:28:37,6.289140e-12,2.070559e-12,1.882640e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv,0.549671,...,7,0.204751,0.099643,0.107535,-0.287430,neutral,1,1,6,9
1,2024-05-01 11:29:06,6.296306e-12,2.071560e-12,1.884595e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv,0.486174,...,39,0.286416,0.106241,0.046128,-0.289120,neutral,9,2,10,1
2,2024-05-01 11:29:07,6.294744e-12,2.074916e-12,1.885256e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv,0.564769,...,23,0.220688,0.262967,0.020917,0.825308,happy,4,3,10,6
3,2024-05-01 11:29:08,6.301253e-12,2.071550e-12,1.884954e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv,0.652303,...,47,0.212123,0.318991,0.041891,-0.265224,neutral,7,3,1,6
4,2024-05-01 11:29:09,6.041612e-12,1.998642e-12,1.852315e-11,sub-NDARAB055BPR_task-DespicableMe_eeg.set,2024-05-01 11:29:00,2024-05-01 11:28:37-04:00,Sinus Rhythm,ecg_2024-05-01.csv,0.476585,...,33,0.203410,0.120590,0.142901,-0.384630,neutral,4,1,9,8


In [21]:
df_merged.to_csv("final_multimodal_dataset.csv", index=False)